

---

## ☸️ **11. Docker with Kubernetes (K8s)**

---

### 🆚 **Container vs Pod**

| Aspect         | Docker Container | Kubernetes Pod               |
| -------------- | ---------------- | ---------------------------- |
| **Unit**       | Single container | One or more containers       |
| **IP Address** | Each container   | Shared IP for all containers |
| **Storage**    | Per container    | Shared volume within Pod     |
| **Usage**      | Standalone apps  | Microservices in K8s cluster |

> 🔑 **In K8s, a Pod is the smallest deployable unit**, not a container.

---

### 🐳➡️☸️ **Using Docker-built Images in Kubernetes**

1. **Build Docker Image**:

   ```bash
   docker build -t myapp:v1 .
   ```

2. **Push to a Registry**:

   ```bash
   docker tag myapp:v1 your_dockerhub/myapp:v1
   docker push your_dockerhub/myapp:v1
   ```

3. **Reference in K8s YAML**:

   ```yaml
   containers:
     - name: myapp
       image: your_dockerhub/myapp:v1
   ```

✅ You can now deploy this image from anywhere using Kubernetes.

---

### 🔁 **Migrating from Docker Compose to K8s YAML**

| Docker Compose | Kubernetes Equivalent         |
| -------------- | ----------------------------- |
| `services:`    | `Deployment`, `Service`       |
| `volumes:`     | `PersistentVolume`, `PVC`     |
| `depends_on:`  | `initContainers`, `lifecycle` |
| `networks:`    | `K8s Service + DNS`           |

🛠️ **Example: Docker Compose**

```yaml
services:
  web:
    image: myapp
    ports:
      - "80:80"
```

🔁 **Equivalent: K8s YAML**

```yaml
apiVersion: v1
kind: Service
metadata:
  name: web-service
spec:
  selector:
    app: myapp
  ports:
    - protocol: TCP
      port: 80
      targetPort: 80
---
apiVersion: apps/v1
kind: Deployment
metadata:
  name: myapp-deployment
spec:
  replicas: 1
  selector:
    matchLabels:
      app: myapp
  template:
    metadata:
      labels:
        app: myapp
    spec:
      containers:
        - name: myapp
          image: your_dockerhub/myapp:v1
```

---

